<a href="https://colab.research.google.com/github/nsjhonson/Vision3D/blob/main/Vision%203D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Step 2: Create a Project Folder and Upload Your Images

In [12]:
# This command creates a folder named 'Vision3D' and a subfolder 'input_images' inside it.
!mkdir -p /content/Vision3D/input_images

print("📁 Folder '/content/Vision3D/input_images' is ready.")
print("➡️ Now, please upload your images to that folder using the file explorer on the left.")

📁 Folder '/content/Vision3D/input_images' is ready.
➡️ Now, please upload your images to that folder using the file explorer on the left.


In [13]:
!mkdir -p /content/Vision3D/input_images
print("📁 Folder ready! Upload 10–20 images (different angles) to /content/Vision3D/input_images")

📁 Folder ready! Upload 10–20 images (different angles) to /content/Vision3D/input_images


 Step 3: Automatically Remove Backgrounds

In [3]:
!pip install rembg pillow tqdm

In [14]:
# The corrected installation command now includes onnxruntime
!pip install rembg onnxruntime pillow tqdm

from rembg import remove
from PIL import Image
import os
from tqdm import tqdm

input_path = "/content/Vision3D/input_images"
output_path = "/content/Vision3D/clean_images"
os.makedirs(output_path, exist_ok=True)

print(f"Processing images from: {input_path}")

for img_name in tqdm(os.listdir(input_path)):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        try:
            input_img_path = os.path.join(input_path, img_name)
            input_img = Image.open(input_img_path)

            output_img = remove(input_img)

            # Save with a .png extension to preserve transparency
            base_name = os.path.splitext(img_name)[0]
            output_img_path = os.path.join(output_path, f"{base_name}.png")
            output_img.save(output_img_path)

        except Exception as e:
            print(f"Could not process {img_name}. Error: {e}")


print("\n✅ Background removal complete! Your clean images are ready in /content/Vision3D/clean_images")

Processing images from: /content/Vision3D/input_images


  0%|          | 0/15 [00:00<?, ?it/s]Downloading data from 'https://github.com/danielgatis/rembg/releases/download/v0.0.0/u2net.onnx' to file '/root/.u2net/u2net.onnx'.

  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  2%|▋                                     | 2.93M/176M [00:00<00:05, 29.3MB/s]
  6%|██▏                                   | 9.98M/176M [00:00<00:03, 53.6MB/s]
 10%|███▊                                  | 17.8M/176M [00:00<00:02, 64.6MB/s]
 14%|█████▍                                | 24.9M/176M [00:00<00:02, 67.5MB/s]
 18%|██████▉                               | 32.0M/176M [00:00<00:02, 68.6MB/s]
 22%|████████▌                             | 39.6M/176M [00:00<00:01, 71.0MB/s]
 27%|██████████                            | 46.7M/176M [00:00<00:01, 69.0MB/s]
 31%|███████████▋                          | 54.1M/176M [00:00<00:01, 70.5MB/s]
 35%|█████████████▏                        | 61.1M/176M [00:00<00:01, 70.6MB/s]
 39%|██████████████▋         


✅ Background removal complete! Your clean images are ready in /content/Vision3D/clean_images


 Step 4: Install Instant-NGP (NeRF)

In [ ]:
# STEP 4: A CLEAN AND COMPLETE INSTALLATION

# Go back to the /content/ directory to start fresh
%cd /content/

# Remove the previous failed attempt
!rm -rf instant-ngp

# Install dependencies and clone the repository recursively
# PLEASE LET THIS COMMAND FINISH. IT CAN TAKE A MINUTE OR TWO.
!apt-get install -qq ninja-build
!git clone --recursive https://github.com/NVlabs/instant-ngp.git
%cd instant-ngp

# Configure the build (without GUI)
# THIS COMMAND CAN ALSO TAKE SOME TIME.
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF

# Now, build the project. This is the longest step (2-5 minutes).
# DO NOT INTERRUPT THIS PROCESS.
!cmake --build build --config RelWithDebInfo -j 4

print("\n✅ Step 4 Complete! Instant-NGP has been installed successfully.")

/content
Cloning into 'instant-ngp'...
remote: Enumerating objects: 4554, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 4554 (delta 91), reused 62 (delta 62), pack-reused 4422 (from 2)
Receiving objects: 100% (4554/4554), 187.25 MiB | 23.88 MiB/s, done.
Resolving deltas: 100% (2896/2896), done.
Submodule 'dependencies/OpenXR-SDK' (https://github.com/KhronosGroup/OpenXR-SDK.git) registered for path 'dependencies/OpenXR-SDK'
Submodule 'dependencies/args' (https://github.com/Taywee/args) registered for path 'dependencies/args'
Submodule 'dependencies/dlss' (https://github.com/NVIDIA/DLSS) registered for path 'dependencies/dlss'
Submodule 'dependencies/glfw' (https://github.com/Tom94/glfw) registered for path 'dependencies/glfw'
Submodule 'dependencies/imgui' (https://github.com/ocornut/imgui.git) registered for path 'dependencies/imgui'
Submodule 'dependencies/pybind11' (https://github.com/Tom94/pybind11) registered fo

 Step 5: Train the 3D Model

In [3]:
# STEP 5: INSTALL REQUIREMENTS AND TRAIN (CORRECTED)

# This shell script sets the environment variable and runs the training
%%shell

# Set the library path to find the GPU drivers
export CUDA_LIB_DIR=$(dirname $(find /usr -name "libcuda.so.1" 2>/dev/null | head -n 1))
export LD_LIBRARY_PATH=$CUDA_LIB_DIR:$LD_LIBRARY_PATH
echo "✅ CUDA Library Path has been set."

# Navigate to the project directory
cd /content/instant-ngp

# Install the required Python packages
pip install -r requirements.txt
echo "✅ Python requirements installed."

# Run the training script
echo "🚀 Starting the training process..."
python scripts/run.py --scene /content/Vision3D/clean_images --train

✅ CUDA Library Path has been set.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 17.5 MB/s eta 0:00:00
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12077 sha256=c890733bccd8b94a462446588b545619220c75a269385183e6b8bd5a3b3747ba
  Stored in directory: /root/.cache/pip/wheels/9e/48/3f/819dbb3f203e96f85b4e73a9edc70326836e339cc79c0a5f38
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=720bf691860951f1f7a0a505ca7e4d19ad576dc738c9b7767ecb36efc1c3a482
  Stored in directory: /root/.cache/pip/wheels/56/1d/e1/e16fb68710a90d89f4762c699f8815956a8bdbcc99e6cfb976
Successfully built commentjson lark-parser
✅ Python requirements installed.
🚀 Starting the training process...
Traceback (most recent call last):
  File "/content/instant-ngp/scripts/r

CalledProcessError: Command '
# Set the library path to find the GPU drivers
export CUDA_LIB_DIR=$(dirname $(find /usr -name "libcuda.so.1" 2>/dev/null | head -n 1))
export LD_LIBRARY_PATH=$CUDA_LIB_DIR:$LD_LIBRARY_PATH
echo "✅ CUDA Library Path has been set."

# Navigate to the project directory
cd /content/instant-ngp

# Install the required Python packages
pip install -r requirements.txt
echo "✅ Python requirements installed."

# Run the training script
echo "🚀 Starting the training process..."
python scripts/run.py --scene /content/Vision3D/clean_images --train
' returned non-zero exit status 1.

Step 6: Export the 3D Model

In [ ]:
!python scripts/export_mesh.py --scene /content/Vision3D/clean_images --output /content/Vision3D/output_model.obj
print("✅ The .obj model has been saved to /content/Vision3D/output_model.obj")

 Step 7: Visualize the 3D Model

In [ ]:
!pip install pythreejs

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display

# This is a basic visualization of a cube.
# Loading and displaying a custom .obj file in Colab with textures can be complex.
# For now, you can download the model and view it with a 3D viewer on your computer.

mesh = Mesh(geometry=BoxGeometry(1,1,1), material=MeshStandardMaterial(color='lightblue'))
camera = PerspectiveCamera(position=[3, 3, 3])
scene = Scene(children=[mesh, AmbientLight(intensity=0.8)])
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=600, height=400)
display(renderer)

Step 8: Download the 3D Model

In [ ]:
from google.colab import files
files.download('/content/Vision3D/output_model.obj')